<a href="https://colab.research.google.com/github/willjhliang/traffic-sign-recognition/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Download dataset from github repo
!git clone --quiet https://github.com/willjhliang/traffic-sign-recognition.git
!mv traffic-sign-recognition/* .
!rm -r traffic-sign-recognition

In [ ]:
from copy import deepcopy
import itertools
from tqdm import tqdm

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn-whitegrid')

from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb

import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils import data
import torchvision

from constants import K, S, class_size, validation_ratio, random_seed
from data import load_data, consolidate_data, split_validation, visualize_data, compare_class_dist
from augment import augment_dataset, visualize_augmentation
from dimensionality_reduction import run_pca, visualize_pca, visualize_pca_per_channel
from torch_utils import load_torch_data, train_model, evaluate_model
from evaluation import generate_confusion_matrix, get_classification_report, generate_class_comparison

# Dataset

In [ ]:
train_data = load_data('data/filtered_images/train')
test_data = load_data('data/filtered_images/test')
labels = pd.read_csv("data/filtered_labels.csv")

In [ ]:
visualize_data(train_data)
compare_class_dist(train_data, test_data)

In [ ]:
original_train_data = deepcopy(train_data)
train_data = augment_dataset(train_data)
compare_class_dist(original_train_data, train_data)

In [ ]:
X_train, X_train_flattened, y_train = consolidate_data(train_data)
X_test, X_test_flattened, y_test = consolidate_data(test_data)

print(f'X_train shape: {X_train.shape}')
print(f'X_train_flattened shape: {X_train_flattened.shape}')
print(f'X_test shape: {X_test.shape}')

# Models

## Dense Neural Network

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(32 * 32 * 3, 128)
        self.layer2 = nn.Linear(128, 64)
        self.out_layer = nn.Linear(64, K)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.out_layer(x)
        return x

## Convolutional Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.batch_norm_1 = nn.BatchNorm2d(32)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.batch_norm_2 = nn.BatchNorm2d(32)
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_3 = nn.BatchNorm2d(64)
        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_4 = nn.BatchNorm2d(64)
        self.dropout_1 = nn.Dropout(0.5)
        self.conv_5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_5 = nn.BatchNorm2d(64)
        self.conv_6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batch_norm_6 = nn.BatchNorm2d(64)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_1 = nn.Linear(4 * 4 * 64, 128)
        self.dropout_3 = nn.Dropout(0.25)
        self.linear_2 = nn.Linear(128, K)

    def forward(self, x):
        x = self.relu(self.batch_norm_1(self.conv_1(x)))
        x = self.relu(self.batch_norm_2(self.conv_2(x)))
        x = self.max_pool2d(x)
        x = self.relu(self.batch_norm_3(self.conv_3(x)))
        x = self.relu(self.batch_norm_4(self.conv_4(x)))
        x = self.dropout_1(x)
        x = self.max_pool2d(x)
        x = self.relu(self.batch_norm_5(self.conv_5(x)))
        x = self.relu(self.batch_norm_6(self.conv_6(x)))
        x = self.dropout_2(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_3(x)
        x = self.linear_2(x)
        return x

# Transfer Learning

In [ ]:
def Resnet():
    model = torchvision.models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, K)
    return model

In [ ]:
def VGG16():
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, K)
    return model

In [ ]:
def EfficientNet():
    model = torchvision.models.efficientnet_b0(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, K)
    return model

In [ ]:
class UpscaleDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, transform):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.numpy()
        img = self.transform(torch.from_numpy(self.X[idx].astype(np.float32)))
        label = torch.tensor(self.y[idx])

        return img, label


def load_vit_data(X_train, y_train, X_test, y_test):
    X_train, X_val, y_train, y_val = split_validation(X_train, y_train)
    transform = torchvision.transforms.Resize((224, 224))
    train_set = UpscaleDataset(X_train, y_train, transform)
    val_set = UpscaleDataset(X_val, y_val, transform)
    test_set = UpscaleDataset(X_test, y_test, transform)
    train_loader = data.DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = data.DataLoader(val_set, batch_size=32, shuffle=True)
    test_loader = data.DataLoader(test_set, batch_size=32, shuffle=True)
    return train_loader, val_loader, test_loader


def VisionTransformer():
    model = torchvision.models.vit_b_16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.heads[-1].in_features
    model.heads[-1] = nn.Linear(num_ftrs, K)
    return model

# Training and Evaluation

In [ ]:
print('========== Dense Neural Network ==========')
train_loader, val_loader, test_loader = load_torch_data(X_train_flattened, y_train, X_test_flattened, y_test)
model = train_model(NN(), train_loader, val_loader, 10, 1e-3)
y_pred_nn, acc = evaluate_model(model, test_loader)
print(f'Test Accuracy: {acc}')

In [ ]:
nn_reportdict, nn_df = get_classification_report(y_test, y_pred_nn, labels)
generate_class_comparison(nn_reportdict, 'f1-score', labels)

In [ ]:
print('========== Convolutional Neural Network ==========')
train_loader, val_loader, test_loader = load_torch_data(X_train, y_train, X_test, y_test)
model = train_model(CNN(), train_loader, val_loader, 10, 1e-3)
y_pred_cnn, accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy}')

In [ ]:
cnn_reportdict, cnn_df = get_classification_report(y_test, y_pred_cnn, labels)
generate_class_comparison(cnn_reportdict, 'f1-score', labels)

In [ ]:
print('========== Transfer Learning Resnet ==========')
train_loader, val_loader, test_loader = load_torch_data(X_train, y_train, X_test, y_test)
model = train_model(Resnet(), train_loader, val_loader, 10, 1e-3)
y_pred_res, accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy}')

In [ ]:
res_reportdict, res_df = get_classification_report(y_test, y_pred_res, labels)
generate_class_comparison(res_reportdict, 'f1-score', labels)

In [ ]:
print('========== Transfer Learning VGG16 ==========')
train_loader, val_loader, test_loader = load_torch_data(X_train, y_train, X_test, y_test)
model = train_model(VGG16(), train_loader, val_loader, 10, 1e-3)
y_pred_vgg, accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy}')

In [ ]:
vgg_reportdict, vgg_df = get_classification_report(y_test, y_pred_vgg, labels)
generate_class_comparison(vgg_reportdict, 'f1-score', labels)

In [ ]:
print('========== Transfer Learning EfficientNet ==========')
train_loader, val_loader, test_loader = load_torch_data(X_train, y_train, X_test, y_test)
model = train_model(EfficientNet(), train_loader, val_loader, 10, 1e-3)
y_pred_eff, accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy}')

In [ ]:
eff_reportdict, eff_df = get_classification_report(y_test, y_pred_eff, labels)
generate_class_comparison(eff_reportdict, 'f1-score', labels)

In [ ]:
print('========== Transfer Learning Vision Transformer ==========')
train_loader, val_loader, test_loader = load_vit_data(X_train, y_train, X_test, y_test)
model = train_model(VisionTransformer(), train_loader, val_loader, 10, 1e-3)
y_pred_vit, accuracy = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy}')

In [ ]:
vit_reportdict, vit_df = get_classification_report(y_test, y_pred_vit, labels)
generate_class_comparison(vit_reportdict, 'f1-score', labels)